# St. Clair County Land Use and Parcel Data Pipeline

In [1]:
!pip3 install Selenium


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

## Extraction

### Download St. Clair Co. Property Tax Inquiry Selected Townships Parcel Listing 

In [4]:
search_pg = "https://stclairil.devnetwedge.com/"
cwd = os.getcwd()

In [5]:
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--headless=new")
prefs = {"download.default_directory": f"{cwd}",
         "safebrowsing.enabled": "false"}
options.add_experimental_option("prefs", prefs)

In [6]:
driver = webdriver.Chrome(options)
driver.implicitly_wait(3)
driver.get(search_pg)

In [7]:
# Click into Advanced Search Tab
advance_search_tab = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//a[@href='#advanced-search']"))
)
advance_search_tab.click()

In [8]:
# Select Townships
township_select = Select(WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "advanced-search-townships"))
))
township_select.select_by_value("02")
township_select.select_by_value("11")
township_select.select_by_value("01")
township_select.select_by_value("06")

In [9]:
# Check All Years Box and Search
all_years_chkbx = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "advanced-search-include-all-years"))
)
form = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//form"))
)
driver.execute_script(f"document.getElementById('advanced-search-include-all-years').click()")
all_years_chkbx.submit()

In [10]:
# Export Results to CSV and Download
export_btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[@href='/Search/ExportClientsListToCSV']"))
)
export_btn.click()

In [11]:
# driver.quit()